In [1]:
import numpy as np
#np.set_printoptions(threshold=np.inf)

world = -1. * np.ones((18,14))
starts = [(17,3), (17,4), (17,5), (17,6)]
goals = [(1,13), (2,13), (3,13), (4,13), (5,13)]

beg_ = [(6,11),(7,9),(8,8),(9,8),(10,8),(11,8),(12,8),(13,7),(14,7),(15,7),(16,7),(17,7)]
end_ = [(0,13),(1,4),(2,0),(3,0),(4,0),(5,0),(6,1),(7,1),(8,1),(9,2),(10,2),(11,3),(12,3),(13,3),(14,3),(15,2),(16,2),(17,2)]

for coup in end_:
    world[coup[0], 0:coup[1]+1] = -10
    
for coup in beg_:
    world[coup[0], coup[1]:14] = -10

for cell in goals:
    world[cell] = -1

print(world)

[[-10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1. -10. -10. -10.

In [154]:
MIN = 1
MAX = 5

def generate_actions():
    actions = []
    for i in range(MAX+1):
        for j in range(MAX+1):
            if MIN <= (i + j) <= MAX:
                actions.append((i,j))
    return actions

def is_alive(state):
    # Out of bounds
    if ((state['y'] < 0 or state['y'] >= world.shape[0]) or 
        (state['x'] < 0 or state['x'] >= world.shape[1])):
        return False
    
    # Goal state
    elif ((state['y'], state['x']) in goals):
        return False
    
    # Obstacle hit
    elif world[state['y'], state['x']] == -10:
        return False
    
    else:
        return True

def move(state, action):
    new_state = state.copy()
    
    # One of the velocity is 0
    if action[0] == 0 or action[1] == 0:        
        # Index of non-zero velocity
        idx = 0 if action[1] == 0 else 1
        
        # Dictionary entry based on idx
        k = 'x' if action[1] == 0 else 'y'
        
        # Simulate trajectory
        for x in range(action[idx]):
            if k == "x":
                new_state[k] += 1
            else:
                new_state[k] -= 1
            
            # Check if alive
            if not is_alive(new_state):
                return new_state
            
    # Diagonal movement
    else:
        if action[0] >= action[1]:
            # Simulate trajectory
            for x in range(action[0]):
                new_state['y'] -= 1
                new_state['x'] += 1

                # Check if alive
                if not is_alive(new_state):
                    return new_state
        else:
            # Simulate trajectory
            for x in range(action[0]):
                new_state['y'] -= 1
                new_state['x'] += 1

                # Check if alive
                if not is_alive(new_state):
                    return new_state
            
            for x in range(action[1] - action[0]):
                new_state['y'] -= 1
                
                # Check if alive
                if not is_alive(new_state):
                    return new_state
    
    return new_state

def monte_carlo():
    # Action-Value table
    Q = np.zeros_like(world)
    
    # Returns dictionary
    R = {}
    
    # Possible actions
    actions = generate_actions()
    
    for _ in range(1000):
        # Start episode 
        alive = True

        # Starting state
        state = {'y': 17, 'x': np.random.randint(0,len(starts))}
        
        while alive:
            # Enconding key
            key = ', '.join(str(x) for x in state.values())
            
            # Check if key not already in dict
            if key not in R.keys():
                R[key] = []
                
            # Action choice
            action = actions[np.random.randint(0, len(actions))]
            
            # Apply move to action
            new_state = move(state, action)
            
            # Update return dict
            R[key].append(world[new_state['y'], new_state['x']])
            
            # Check if new state valid
            if is_alive(new_state):
                # Update state
                state = new_state
            else:
                alive = False
        
        # Update action value states
        for state in R.keys():
            # Get indices
            idxs = state.split(",")
            
            # Compute average
            Q[int(idxs[0]), int(idxs[1])] = np.mean(R[state])
        
    print(Q)

In [155]:
monte_carlo()

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
    0.         -10.         -10.         -10.          -7.
  -10.         -10.          -4.           0.        ]
 [  0.           0.           0.           0.           0.
    0.         -10.          -1.          -5.5          0.
   -7.          -2.8         -1.           0.        ]
 [  0.           0.           0.           0.           0.
    0.          -1.          -4.         -10.          -5.5
   -3.25        -1.          -3.25         0.        ]
 [  0.           0.           0.           0.          -1.
   -4.           0.          -4.6         -1.          -4.
   -5.5         -1.          -1.           0.        ]
 [  0.           0.           0.           0.          -1.
    0.          -2.5         -3.25        -2.5         -2.125
   -1.       